In [1]:
from langchain_community.document_loaders import TextLoader #this is the TextLoader; it takes the raw text the text of the book descriptions and convert it into a format that Langchain can work with.

from langchain_text_splitters import CharacterTextSplitter #this is going to split the whole document containing all the descriptions into meaningful chunks. Here these chunks would be a meaningful description of each book.

from langchain_openai import OpenAIEmbeddings #converting those chunks into document embeddings, using the OpenAI embedding method.

from langchain_chroma import Chroma #storing the embeddings in a vector database using chroma

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [12]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [13]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = " ",
                                   index = False,
                                   header = False)

In [14]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator=" ")
documents = text_splitter.split_documents(raw_documents)

In [16]:
documents

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redempt

In [55]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of')

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load a free, local embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store in ChromaDB
db_books = Chroma.from_documents(
    documents,
    embedding=embedding_model
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Ragha\PycharmProjects\book-recommender\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ragha\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [64]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='science fiction gain wide public attention, and left a lasting impression upon society. The same writers formed the mold for the next three decades of science fiction, and much of their writing remains as fresh today as it was then."\n"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes p

In [66]:
print(docs[0].page_content)  # Check the full content
print(docs[0].page_content.split())  # Check split parts

science fiction gain wide public attention, and left a lasting impression upon society. The same writers formed the mold for the next three decades of science fiction, and much of their writing remains as fresh today as it was then."
"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning
['science', 'fiction', 'gain', 'wide', 'public', 'attention,', 'and', 'left', 'a', 'lasting', 'impression', 'upon', 'society.', 'The', 'same', 'writers', 'formed', 'the', 'mold', 'for', 'the', 'next', 'three', 'decades', 'of', 'science', 'fiction,', 'and', 'much', 'of', 'their', 'writing', 'remains', 'as', 'fresh', 'today', 'as', 'it', 'was', 'then."', '"9780786808069', 'Children', 'will', 'discover', 'the', 'exciting', 'world', 'of', 'their', 'own', 'backyard', 'in', 'this', 'introduction', 't

In [67]:
# If content is "Category: science, ISBN13: 1234567890123":
isbn_str = docs[0].page_content.split()[3].strip()  # Extract "1234567890123"

In [69]:
import re

# Extract the ISBN13 safely
page_content = docs[0].page_content
isbn_match = re.search(r'\d{13}', page_content)

if isbn_match:
    isbn13 = int(isbn_match.group())
    result = books[books["isbn13"] == isbn13]
else:
    print("ISBN13 not found in the document.")
    result = pd.DataFrame()  # Handle missing ISBN case

In [40]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(documents, embedding=embedding_model)

In [72]:
import re

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    # Fetch recommendations via similarity search
    recs = db_books.similarity_search(query, k=50)

    # Extract ISBN13 values safely
    isbn_list = []
    for doc in recs:
        # Use regex to find the first 13-digit number in the document's content
        isbn_match = re.search(r'\d{13}', doc.page_content)
        if isbn_match:
            try:
                isbn = int(isbn_match.group())
                isbn_list.append(isbn)
            except ValueError:
                # Skip invalid ISBNs (unlikely if regex matches \d{13})
                continue

    # Return top_k results from the filtered DataFrame
    return books[books["isbn13"].isin(isbn_list)].head(top_k)

In [73]:
# Get recommendations for a query
result = retrieve_semantic_recommendations("A book to teach children about nature")
print(result)

             isbn13      isbn10                          title  \
325   9780060973339  0060973331                Fear of Falling   
363   9780061120077  0061120073       A Tree Grows in Brooklyn   
384   9780061145391  0061145394             The Gabriel Hounds   
394   9780061312113  0061312118       The Blue and Brown Books   
436   9780064471961  0064471969        Shade's Children (rack)   
992   9780194230353  019423035X  The Hound of the Baskervilles   
1639  9780374422080  0374422087         Everything on a Waffle   
1641  9780374517885  0374517886                       Sammlung   
1881  9780385732567  0385732562                 Gathering Blue   
1886  9780393011838  0393011836    What Einstein Told His Cook   

                              authors        categories  \
325                Barbara Ehrenreich    Social Science   
363                       Betty Smith           Fiction   
384                      Mary Stewart           Fiction   
394               Ludwig Wittgenstein